https://github.com/dbouchabou/Fully-Convolutional-Network-Smart-Homes/tree/master

In [1]:
from google.colab import drive
drive.mount('/content/drive' , force_remount = True)

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### MHEALTH

In [75]:
cd "/content/drive/MyDrive/MHEALTHDATASET/"

/content/drive/MyDrive/MHEALTHDATASET


In [76]:
df_mhealth = pd.read_pickle('mhealth_for_lstm.pkl')

In [77]:
data = df_mhealth
grouped_data = df_mhealth.groupby('subject')
train_data = pd.DataFrame()
val_data = pd.DataFrame()
test_data = pd.DataFrame()

for _, group in grouped_data:
    sorted_group = group.sort_index()

    total_samples = len(sorted_group)
    train_size = int(0.6 * total_samples)
    val_size = int(0.2 * total_samples)
    test_size = int(0.2 * total_samples)

    train_group = sorted_group.iloc[val_size:val_size+train_size]
    val_group = sorted_group.iloc[:val_size]
    test_group = sorted_group.iloc[val_size + train_size:]

    train_data = pd.concat([train_data, train_group])
    test_data = pd.concat([test_data, test_group])
    val_data = pd.concat([val_data, val_group])

In [78]:
data = df_mhealth
grouped_data = df_mhealth.groupby('subject')
train_data = pd.DataFrame()
val_data = pd.DataFrame()
test_data = pd.DataFrame()

for _, group in grouped_data:
    sorted_group = group.sort_index()

    total_samples = len(sorted_group)
    train_size = int(0.6 * total_samples)
    val_size = int(0.2 * total_samples)
    test_size = int(0.2 * total_samples)

    train_group = sorted_group.iloc[:train_size]
    val_group = sorted_group.iloc[train_size:train_size+val_size]
    test_group = sorted_group.iloc[val_size + train_size:]

    train_data = pd.concat([train_data, train_group])
    test_data = pd.concat([test_data, test_group])
    val_data = pd.concat([val_data, val_group])

In [79]:
#scaling
from sklearn.preprocessing import StandardScaler
numerical_columns = ['acc_ch_x','acc_ch_y','acc_ch_z','ecg_sig_1', 'ecg_sig_2','acc_la_x','acc_la_y','acc_la_z','gyr_la_x','gyr_la_y','gyr_la_z','mag_la_x','mag_la_y','mag_la_z','acc_rw_x','acc_rw_y','acc_rw_z','gyr_rw_x','gyr_rw_y','gyr_rw_z','mag_rw_x','mag_rw_y','mag_rw_z',]
scaler = StandardScaler()

scaler.fit(train_data[numerical_columns])
train_data[numerical_columns] = scaler.transform(train_data[numerical_columns])
val_data[numerical_columns] = scaler.transform(val_data[numerical_columns])
test_data[numerical_columns] = scaler.transform(test_data[numerical_columns])

In [80]:
def segment_activities(df):
    activitiesSeq = []

    ponentialIndex = df.activity.ne(df.activity.shift())

    ii = np.where(ponentialIndex == True)[0]

    for i,end in enumerate(ii):
        if i > 0 :

          dftmp = df[ii[i-1]:end]
          activitiesSeq.append(dftmp)
    return activitiesSeq

train_activitySequences = segment_activities(train_data)
val_activitySequences = segment_activities(val_data)
test_activitySequences = segment_activities(test_data)

In [81]:
def generate_sentence(df2):
    sentence = ""

    #columns for which to retain values
    columns_of_interest = ['acc_ch_x', 'acc_ch_y', 'acc_ch_z', 'ecg_sig_1', 'ecg_sig_2',
       'acc_la_x', 'acc_la_y', 'acc_la_z', 'gyr_la_x', 'gyr_la_y', 'gyr_la_z',
       'mag_la_x', 'mag_la_y', 'mag_la_z', 'acc_rw_x', 'acc_rw_y', 'acc_rw_z',
       'gyr_rw_x', 'gyr_rw_y', 'gyr_rw_z', 'mag_rw_x', 'mag_rw_y', 'mag_rw_z',
       'activity', 'subject']

    #iterate over columns
    for column in columns_of_interest:
        #column value
        value = df2[column].values[0]  # Directly access the single value in the column

        #column name and val
        sentence += "{}{}".format(column, value)

        #space if not last col
        if column != columns_of_interest[-1]:
            sentence += " "

    return sentence

In [82]:
def sequencesToSentences(activitySequences):
	sentences = []
	label_sentences = []

	for i in range(len(activitySequences)):

		sentence = generate_sentence(activitySequences[i])

		sentences.append(sentence)
		label_sentences.append(activitySequences[i].activity.values[0])

	return sentences, label_sentences

In [83]:
train_sentences, train_label_sentences = sequencesToSentences(train_activitySequences)
val_sentences, val_label_sentences = sequencesToSentences(val_activitySequences)
test_sentences, test_label_sentences = sequencesToSentences(test_activitySequences)

tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
combined_sentences = train_sentences + val_sentences + test_sentences
tokenizer.fit_on_texts(combined_sentences)

# Tokenize train, validation, and test sets
train_indexed_sentences = tokenizer.texts_to_sequences(train_sentences)
val_indexed_sentences = tokenizer.texts_to_sequences(val_sentences)
test_indexed_sentences = tokenizer.texts_to_sequences(test_sentences)

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7b06f8421a20>
Traceback (most recent call last):
  File "/usr/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


In [84]:
def slidingWindow(sequence,winSize,step=1):

    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")

    numOfChunks = int(((len(sequence)-winSize)/step)+1)

    if winSize > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i in range(0,numOfChunks*step,step):
            yield sequence[i:i+winSize]

In [85]:
X_train_windowed = []
Y_train_windowed = []
winSize = 50
step = 1

for i,s in enumerate(train_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_train_windowed.append(chunk)
		Y_train_windowed.append(train_label_sentences[i])

X_val_windowed = []
Y_val_windowed = []
winSize = 50
step = 1

for i,s in enumerate(val_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_val_windowed.append(chunk)
		Y_val_windowed.append(val_label_sentences[i])


X_test_windowed = []
Y_test_windowed = []
winSize = 50
step = 1

for i,s in enumerate(test_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_test_windowed.append(chunk)
		Y_test_windowed.append(test_label_sentences[i])

In [86]:
train_padded_windows = pad_sequences(X_train_windowed)
test_padded_windows = pad_sequences(X_test_windowed)
val_padded_windows = pad_sequences(X_val_windowed)

Y_train_windowed = np.array(Y_train_windowed)
Y_test_windowed = np.array(Y_test_windowed)
Y_val_windowed = np.array(Y_val_windowed)

In [87]:
x_train = train_padded_windows
y_train = Y_train_windowed
x_test = test_padded_windows
y_test = Y_test_windowed
x_val = val_padded_windows
y_val = Y_val_windowed

In [88]:
from scipy.stats import entropy

js_divergences = np.zeros(x_train.shape[1])
for i in range(x_train.shape[1]):
    # Compute histograms with the same bins for both X_train and X_test
    bins = max(len(np.unique(x_train[:, i])), len(np.unique(x_test[:, i])))
    p, _ = np.histogram(x_train[:, i], bins=bins, density=True)
    q, _ = np.histogram(x_test[:, i], bins=bins, density=True)
    m = 0.5 * (p + q)
    js_divergences[i] = 0.5 * (entropy(p, m) + entropy(q, m))

avg_js_divergence = np.mean(js_divergences)
print("Average Jensen-Shannon divergence:", avg_js_divergence)

Average Jensen-Shannon divergence: 0.22045810936700894


### PAMAP2

In [3]:
cd "/content/drive/MyDrive/PAMAP2_Dataset"

/content/drive/MyDrive/PAMAP2_Dataset


In [4]:
data = pd.read_pickle('pamap2_for_lstm.pkl')

Take last

#### ~ 200k samples per id

In [5]:
columns_of_interest = ['time_stamp', 'activity_id','heart_rate', 'hand_temperature','hand_3D_acceleration_16_x', 'hand_3D_acceleration_16_y','hand_3D_acceleration_16_z', 'hand_3D_gyroscope_x', 'hand_3D_gyroscope_y', 'hand_3D_gyroscope_z','chest_3D_acceleration_16_x', 'chest_3D_acceleration_16_y','chest_3D_acceleration_16_z','chest_3D_gyroscope_x', 'chest_3D_gyroscope_y', 'chest_3D_gyroscope_z','ankle_3D_acceleration_16_x', 'ankle_3D_acceleration_16_y','ankle_3D_acceleration_16_z','ankle_3D_gyroscope_x', 'ankle_3D_gyroscope_y', 'ankle_3D_gyroscope_z', 'id']

data = data[columns_of_interest]
data = data[data['id'] != 109]

In [55]:
from sklearn.model_selection import train_test_split

grouped_data = data.groupby('id')
train_data = pd.DataFrame()
test_data = pd.DataFrame()
val_data = pd.DataFrame()

for _, group in grouped_data:
    sorted_group = group.sort_values(by='time_stamp', ascending=True)

    # Calculate the size of training data dynamically based on the first 60% of rows
    total_samples = len(sorted_group)
    train_size = int(0.6 * total_samples)
    val_size = int(0.2 * total_samples)
    test_size = int(0.2 * total_samples)

    train_group = sorted_group.iloc[:train_size]
    val_group = sorted_group.iloc[train_size:val_size]
    test_group = sorted_group.iloc[val_size + train_size:]

    train_data = pd.concat([train_data, train_group])
    val_data = pd.concat([val_data, val_group])
    test_data = pd.concat([test_data, test_group])

scale

In [56]:
train_activity_id = train_data['activity_id']
train_sub_id = train_data['id']
test_activity_id = test_data['activity_id']
test_sub_id = test_data['id']
val_activity_id = val_data['activity_id']
val_sub_id = val_data['id']

train_df_without_id = train_data.drop(columns=['activity_id', 'id'])
test_df_without_id = test_data.drop(columns=['activity_id', 'id'])
val_df_without_id = val_data.drop(columns=['activity_id', 'id'])

In [57]:
from sklearn.preprocessing import StandardScaler
numerical_columns = train_df_without_id.select_dtypes(include=['number']).columns

scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_df_without_id[numerical_columns])
train_scaled_df = pd.DataFrame(train_scaled, columns=numerical_columns, index=train_df_without_id.index)

test_scaled = scaler.transform(test_df_without_id[numerical_columns])
test_scaled_df = pd.DataFrame(test_scaled, columns=numerical_columns, index=test_df_without_id.index)

val_scaled = scaler.transform(val_df_without_id[numerical_columns])
val_scaled_df = pd.DataFrame(val_scaled, columns=numerical_columns, index=val_df_without_id.index)

train_scaled_df = pd.concat([train_activity_id, train_sub_id, train_scaled_df], axis=1)
test_scaled_df = pd.concat([test_activity_id, test_sub_id, test_scaled_df], axis=1)
val_scaled_df = pd.concat([val_activity_id, val_sub_id, val_scaled_df], axis=1)

In [58]:
def segment_activities(df):
    activitiesSeq = []

    ponentialIndex = df.activity_id.ne(df.activity_id.shift())

    ii = np.where(ponentialIndex == True)[0]

    for i,end in enumerate(ii):
        if i > 0 :

          dftmp = df[ii[i-1]:end]
          activitiesSeq.append(dftmp)
    return activitiesSeq

In [59]:
train_activitySequences = segment_activities(train_data)
val_activitySequences = segment_activities(val_data)
test_activitySequences = segment_activities(test_data)

#### with my columns

In [60]:
def generate_sentence(df2):
    sentence = ""

    # Define the column names you want to extract values from
    columns_of_interest = ['time_stamp', 'activity_id','heart_rate', 'hand_temperature','hand_3D_acceleration_16_x', 'hand_3D_acceleration_16_y','hand_3D_acceleration_16_z', 'hand_3D_gyroscope_x', 'hand_3D_gyroscope_y', 'hand_3D_gyroscope_z','chest_3D_acceleration_16_x', 'chest_3D_acceleration_16_y','chest_3D_acceleration_16_z','chest_3D_gyroscope_x', 'chest_3D_gyroscope_y', 'chest_3D_gyroscope_z','ankle_3D_acceleration_16_x', 'ankle_3D_acceleration_16_y','ankle_3D_acceleration_16_z','ankle_3D_gyroscope_x', 'ankle_3D_gyroscope_y', 'ankle_3D_gyroscope_z', 'id']

    # Iterate over the columns of interest
    for column in columns_of_interest:
        # Get the value for the current column
        value = df2[column].values[0]  # Directly access the single value in the column

        # Add column name and value to the sentence
        sentence += "{}{}".format(column, value)

        # Add a space if it's not the last column
        if column != columns_of_interest[-1]:
            sentence += " "

    return sentence

In [61]:
def sequencesToSentences(activitySequences):
	sentences = []
	label_sentences = []

	for i in range(len(activitySequences)):

		sentence = generate_sentence(activitySequences[i])

		sentences.append(sentence)
		label_sentences.append(activitySequences[i].activity_id.values[0])

	return sentences, label_sentences

In [62]:
train_sentences, train_label_sentences = sequencesToSentences(train_activitySequences)
val_sentences, val_label_sentences = sequencesToSentences(val_activitySequences)
test_sentences, test_label_sentences = sequencesToSentences(test_activitySequences)

#### use 1 tokenizer for all

In [63]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
combined_sentences = train_sentences + val_sentences + test_sentences
tokenizer.fit_on_texts(combined_sentences)

# Tokenize train, validation, and test sets
train_indexed_sentences = tokenizer.texts_to_sequences(train_sentences)
val_indexed_sentences = tokenizer.texts_to_sequences(val_sentences)
test_indexed_sentences = tokenizer.texts_to_sequences(test_sentences)

In [64]:
def slidingWindow(sequence,winSize,step=1):

    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")

    numOfChunks = int(((len(sequence)-winSize)/step)+1)

    # Do the work
    if winSize > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i in range(0,numOfChunks*step,step):
            yield sequence[i:i+winSize]

In [65]:
winSize = 50
step = 1

X_train_windowed = []
Y_train_windowed = []
for i,s in enumerate(train_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_train_windowed.append(chunk)
		Y_train_windowed.append(train_label_sentences[i])

X_val_windowed = []
Y_val_windowed = []
for i,s in enumerate(val_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_val_windowed.append(chunk)
		Y_val_windowed.append(val_label_sentences[i])


X_test_windowed = []
Y_test_windowed = []
for i,s in enumerate(test_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_test_windowed.append(chunk)
		Y_test_windowed.append(test_label_sentences[i])

In [66]:
train_padded_windows = pad_sequences(X_train_windowed)
test_padded_windows = pad_sequences(X_test_windowed)
val_padded_windows = pad_sequences(X_val_windowed)

Y_train_windowed = np.array(Y_train_windowed)
Y_test_windowed = np.array(Y_test_windowed)
Y_val_windowed = np.array(Y_val_windowed)

In [67]:
x_train = train_padded_windows
y_train = Y_train_windowed
x_test = test_padded_windows
y_test = Y_test_windowed
x_val = val_padded_windows
y_val = Y_val_windowed

In [68]:
from scipy.stats import entropy

js_divergences = np.zeros(x_train.shape[1])
for i in range(x_train.shape[1]):
    # Compute histograms with the same bins for both X_train and X_test
    bins = max(len(np.unique(x_train[:, i])), len(np.unique(x_test[:, i])))
    p, _ = np.histogram(x_train[:, i], bins=bins, density=True)
    q, _ = np.histogram(x_test[:, i], bins=bins, density=True)
    m = 0.5 * (p + q)
    js_divergences[i] = 0.5 * (entropy(p, m) + entropy(q, m))

avg_js_divergence = np.mean(js_divergences)
print("Average Jensen-Shannon divergence:", avg_js_divergence)

Average Jensen-Shannon divergence: 0.12993332932141433


### main

#### FCN embedded

In [ ]:
pip install tensorflow

In [69]:
class LSTMModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm1 = tf.keras.layers.LSTM(hidden_dim, return_sequences=True)
        self.dropout = tf.keras.layers.Dropout(0.5)  # Example dropout rate
        self.lstm2 = tf.keras.layers.LSTM(hidden_dim)
        self.fc = tf.keras.layers.Dense(output_dim, activation='softmax')

    def call(self, inputs):
        embedded = self.embedding(inputs)
        lstm_out1 = self.lstm1(embedded)
        lstm_out2 = self.lstm2(self.dropout(lstm_out1))
        output = self.fc(lstm_out2)
        return output

In [70]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
hidden_dim = 128
output_dim = 25

In [71]:
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=20, validation_data=(x_val, y_val))

Epoch 1/3
265/265 [==============================] - 42s 135ms/step - loss: 2.4133 - accuracy: 0.1816 - val_loss: 2.6937 - val_accuracy: 0.1790
Epoch 2/3
265/265 [==============================] - 36s 134ms/step - loss: 1.4233 - accuracy: 0.4688 - val_loss: 1.5773 - val_accuracy: 0.4356
Epoch 3/3
265/265 [==============================] - 43s 163ms/step - loss: 0.7029 - accuracy: 0.7485 - val_loss: 1.2875 - val_accuracy: 0.6155


#### predict

In [72]:
predictions = model.predict(x_test)

39/39 [==============================] - 3s 46ms/step


In [73]:
import numpy as np

class_predictions = np.argmax(predictions, axis=1)

In [74]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, class_predictions)
print("Accuracy:", accuracy)

Accuracy: 0.40468497576736673
